# OC4920 - GEOSTROPHY

## Outline

Good morning everyone!

So yesterday, we talked about geostrophic velocities and two different ways of estimating them. The first from horizontal sea surface height gradients (horizontal along height/depth), the second from horizontal density gradients (horizontal along isobars).

The first method provides absolute mean velocities for the entire water column, *ie*. the barotropic component of the geostrophic velocity.

The second method provides *no* velocity measurements but provides the vertical structure of the currents, $\delta u / \delta z$, also called vertical shear.

You will by now have read through Estel's paper and have an understanding of why we may want to measure flows at this location. Persian Gulf Water flows out of the Strait of Hormuz as a gravity current, rapidly adjusting down the slope, and flowing out into the Oxygen Minimum Zone. The current really stands out with a much warmer, more saline and oxygenated signature.

One aspect that emerges from the paper is the huge variability in PGW concentration and transport, both intraseasonally and interannually. We finish the paper with a first estimate of transport, but also a clear understanding that we have **not** resolved the variability in the system. To do so, we need more measurements over a longer period of time.

From this emerges a simple question: does PGW transport have a surface signature? We know the watermass is not visible at the surface, but is the dominant mode of transport one that aligns with the barotropic velocity? Can we derive a proxy for PGW transport from sea surface heigh anomalies? (Ok, that's more like 3 questions, but you get what I mean...)

For the overachievers among you, the full dataset is available here ([10.5281/zenodo.10075774](https://zenodo.org/records/10075774)) - to be clear, I do not expect you to answer the question above, but there's nothing stopping us from starting the process.

Work together, brainstorm ideas, share code, use ChatGPT, have fun.

## Exercises

In [ ]:
# Imports
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

### Practical 1 (Tuesday)
So the steps to this process will be:
- Obtain sea level anomaly (SLA) data from Copernicus Marine Services along the glider transect. Plot it.
- Calculate geostrophic velocities from SLA using the equation in lecture one. Plot it.
- As this is a learning exercise, we'll also download geostrophic velocities from CMEMS.
- Compare your velocities to the CMEMS velocities. Plot this too.
- Now download an amount of SLA data that you think will be representative of longer term variability. Surprise - plot this too!
- For assessment: 
    - describe what differences exist between both datasets (if any), 
    - what you think the source of these differences is. 
    - Briefly descripe how variable barotropic flows are across the transect location.
    - What assumptions did you make, or shortcuts did you take to calculate these velocities. Are they valid?
    > (Transect coordinates are :  24.05°N, 57.5°E, to 24.5°N, 57.8°E)

In [ ]:
# code

In [ ]:
# code

### Practical 2 (Wednesday)

Today, we collect the glider data ([link here](https://gunet-my.sharepoint.com/:x:/g/personal/bastien_queste_gu_se/EZvSd5h0dwtGq3m3F2ceFL8BaspXJ23Uix_Ie8mWX7u5kg?e=QhY2Tz)) and calculate geostrophic velocities. For this practical, I am giving you only one glider mission. All datasets are provided in the Zenodo link, but the glider data is ~9GB - somewhat over the top for a practical session.

- Take the glider timeseries data, notice how data is irregularly spaced in space and the glider deviates from transect.
- Create gridded sections of glider density data - think about how you might project the data onto a straight transect (you may want to brainstorm as a group there).
- Think about if any filtering (smoothing) is necessary?
- Calculate horizontal distances to get horizontal density gradients (along isobars).
- Calculate vertical shear and integrate vertically to get unreferenced geostrophic velocity profiles (code it up by hand, no TEOS-10 shortcuts please - to be honest though, it's not much of a shortcut).
- Create referenced geostrophic velocity sections using altimetry data.
- For assessment:
    - How does it compare to selecting a level of no motion (say at depth, or near the seabed)?

In [ ]:
# Here's a useful gridding function that is quick and generally painfree
def grid2d(x, y, v, xi=1, yi=1, fn='median'):
    """
    Quick data binning function relying on pandas.
    x,y,v are flat np.arrays of x, y coordinates and values at these points.
    xi and yi are either np.arrays of bins to be binned into, or the spacing used between min and max of x or y respectively.
    fn defines the function applied to all points from v that fall into the same bin.
    """
    if np.size(xi) == 1:
        xi = np.arange(np.nanmin(x), np.nanmax(x) + xi, xi)
    if np.size(yi) == 1:
        yi = np.arange(np.nanmin(y), np.nanmax(y) + yi, yi)

    raw = pd.DataFrame({'x': x, 'y': y, 'v': v}).dropna()

    grid = np.full([np.size(yi), np.size(xi)], np.nan)

    raw['xbins'], xbin_iter = pd.cut(raw.x, xi, retbins=True, labels=False, right=False)
    raw['ybins'], ybin_iter = pd.cut(raw.y, yi, retbins=True, labels=False, right=False)

    _tmp = raw.groupby(['xbins', 'ybins'])['v'].agg(fn)
    grid[
        _tmp.index.get_level_values(1).astype(int),
        _tmp.index.get_level_values(0).astype(int),
    ] = _tmp.values

    XI, YI = np.meshgrid(xi, yi, indexing='ij')
    return grid, XI.T, YI.T

In [ ]:
data = pd.read_table('glider_data_subset.csv', delimiter=',', parse_dates=['time',])
data.describe()

In [ ]:
# code

In [ ]:
# code

### Practical 3 (Thursday)

Practical 3 takes it beyond geostrophic velocities. Today, we use direct measurements of velocity using the ADCP mounted to the glider ([ADCP data file 1](https://gunet-my.sharepoint.com/:u:/g/personal/bastien_queste_gu_se/Ea_le-4JRtZMnNyCJHpwyB0Bx8F2VvCamlLG4kFnBd-uMQ?e=bvgEId) and [ADCP data file 2](https://gunet-my.sharepoint.com/:u:/g/personal/bastien_queste_gu_se/EXOWr8VDhAFIg-x9TXxKCMMBMJWBO9xqn4Ee6GAe8lVbeA?e=DOwvP8)).
Install the gliderad2cp toolbox:
> pip install --user gliderad2cp

- Follow instructions to use the gliderad2cp toolbox using guidance from https://github.com/bastienqueste/gliderad2cp/tree/main and https://www.flow-lab.org/gliderad2cp/.
- You will need to provide latitudes and longitudes for the start and end of each dive. The `dead_reckoning` variable indicates if GPS coordinates are from the dead reckoning model (True) or from GPS readings (False). I provide example code below:
- Assessment:
    - Critically assess the ADCP data.
    - How does it compare to the geostrophic velocities?
    - Suggest a way of referencing geostrophic velocities using the ADCP data
    - Where do ADCP data and geostrophic velocities differ most? What can that tell us?

In [ ]:
gps_predive = []
gps_postdive = []

dives = np.round(np.unique(data.dive_num))

_idx = np.arange(len(data.dead_reckoning.values))
for dn in dives:
    _gd = (data.dive_num.values == dn).astype('float')
    _gd[_gd < 1] = np.nan
    
    _dp = (data.pressure.values > 10).astype('float')
    _dp[_dp < 1] = np.nan

    if all(np.isnan(_gd)):
        plt.plot(_dr)
        continue

    first = int(np.nanmin(_idx * _gd))+20
    last  = int(np.nanmin(_idx * _gd * _dp))

        
    # print(first,last)
    gps_postdive.append(np.array([data.time.values[first], data.longitude.values[first], data.latitude.values[first]]))
    gps_predive.append(np.array([data.time.values[last], data.longitude.values[last], data.latitude.values[last]]))

gps_predive = np.vstack(gps_predive)
gps_postdive = np.vstack(gps_postdive)

In [ ]:
from gliderad2cp import process_currents, process_shear, process_bias, tools

options = tools.get_options(xaxis=1, yaxis=2)

In [ ]:
ds_adcp = process_shear.process('*.nc', data, options)
currents, DAC = process_currents.process(ds_adcp, gps_predive, gps_postdive, options)
# currents = process_bias.process(currents,options)

In [ ]:
# code

In [ ]:
# code